In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework') \
    .getOrCreate()

22/02/28 16:55:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/28 16:55:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## 1

In [3]:
spark.version

'3.0.3'

## 2

In [4]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [5]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/raw/fhv/fhvhv_tripdata_2021-02.csv')

In [7]:
df.repartition(24) \
    .write.parquet('data/pq/fhvhv/2021/02/')

In [27]:
!ls -lh data/pq/fhvhv/2021/02/

total 208M
-rw-r--r-- 1 rasid rasid    0 Feb 28 16:31 _SUCCESS
-rw-r--r-- 1 rasid rasid 8.7M Feb 28 16:31 part-00000-da6813d3-775b-4455-8dca-da327c01fb9d-c000.snappy.parquet
-rw-r--r-- 1 rasid rasid 8.7M Feb 28 16:31 part-00001-da6813d3-775b-4455-8dca-da327c01fb9d-c000.snappy.parquet
-rw-r--r-- 1 rasid rasid 8.7M Feb 28 16:31 part-00002-da6813d3-775b-4455-8dca-da327c01fb9d-c000.snappy.parquet
-rw-r--r-- 1 rasid rasid 8.7M Feb 28 16:31 part-00003-da6813d3-775b-4455-8dca-da327c01fb9d-c000.snappy.parquet
-rw-r--r-- 1 rasid rasid 8.7M Feb 28 16:31 part-00004-da6813d3-775b-4455-8dca-da327c01fb9d-c000.snappy.parquet
-rw-r--r-- 1 rasid rasid 8.7M Feb 28 16:31 part-00005-da6813d3-775b-4455-8dca-da327c01fb9d-c000.snappy.parquet
-rw-r--r-- 1 rasid rasid 8.7M Feb 28 16:31 part-00006-da6813d3-775b-4455-8dca-da327c01fb9d-c000.snappy.parquet
-rw-r--r-- 1 rasid rasid 8.7M Feb 28 16:31 part-00007-da6813d3-775b-4455-8dca-da327c01fb9d-c000.snappy.parquet
-rw-r--r-- 1 rasid rasid 8.7M Feb 28 16:31 part-0

## 3

In [8]:
df_pq = spark.read.parquet('data/pq/fhvhv/2021/02/')

df_pq.show(4)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
only showing top 4 rows



In [9]:
df_pq_date = df_pq.withColumn('pickup_date', F.to_date(df_pq.pickup_datetime)) \
    .withColumn('dropoff_date',  F.to_date(df_pq.dropoff_datetime))

In [10]:
df_pq_date.registerTempTable('fhv_data')

In [14]:
spark.sql("""
SELECT
    count(*)
FROM
    fhv_data
WHERE
    pickup_date = '2021-02-15'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



## 4

In [17]:
df_pq_date.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- dropoff_date: date (nullable = true)



In [43]:
df2_time = df_pq_date.withColumn('DurationInSeconds',df_pq_date.dropoff_datetime.cast("long") - df_pq_date.pickup_datetime.cast("long"))
df2_time.select('pickup_date', 'DurationInSeconds').show(4)

+-----------+-----------------+
|pickup_date|DurationInSeconds|
+-----------+-----------------+
| 2021-02-06|             1319|
| 2021-02-05|             1130|
| 2021-02-04|             1484|
| 2021-02-03|             1375|
+-----------+-----------------+
only showing top 4 rows



In [46]:
df2_time.registerTempTable('fhv_data_duration')
spark.sql("""
SELECT
    pickup_date,
    DurationInSeconds
FROM
    fhv_data_duration
ORDER BY
    DurationInSeconds DESC
""").show(1)

+-----------+-----------------+
|pickup_date|DurationInSeconds|
+-----------+-----------------+
| 2021-02-11|            75540|
+-----------+-----------------+
only showing top 1 row



## 5

In [49]:
spark.sql("""
SELECT
    dispatching_base_num,
    count(dispatching_base_num) as frequency
FROM
    fhv_data_duration
GROUP BY 
    dispatching_base_num
ORDER BY 
    2 DESC
""").show()

+--------------------+---------+
|dispatching_base_num|frequency|
+--------------------+---------+
|              B02510|  3233664|
|              B02764|   965568|
|              B02872|   882689|
|              B02875|   685390|
|              B02765|   559768|
|              B02869|   429720|
|              B02887|   322331|
|              B02871|   312364|
|              B02864|   311603|
|              B02866|   311089|
|              B02878|   305185|
|              B02682|   303255|
|              B02617|   274510|
|              B02883|   251617|
|              B02884|   244963|
|              B02882|   232173|
|              B02876|   215693|
|              B02879|   210137|
|              B02867|   200530|
|              B02877|   198938|
+--------------------+---------+
only showing top 20 rows



## 6

In [58]:
df_zone_pd = pd.read_csv('taxi+_zone_lookup.csv')
df_zone_pd.dtypes

LocationID       int64
Borough         object
Zone            object
service_zone    object
dtype: object

In [60]:
schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.TimestampType(), True)
])

In [65]:
df_zone = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('taxi+_zone_lookup.csv')
df_zone.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|        null|
|         2|       Queens|         Jamaica Bay|        null|
|         3|        Bronx|Allerton/Pelham G...|        null|
|         4|    Manhattan|       Alphabet City|        null|
|         5|Staten Island|       Arden Heights|        null|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [63]:
df_zone.registerTempTable('zone')

In [78]:
spark.sql("""
SELECT 
    loc_pair,
    COUNT(loc_pair) AS frequency
FROM (
    SELECT
        PULocationID,
        pickup_zone.Zone PULocation,
        DOLocationID,
        dropoff_zone.Zone DOLocation,
        CONCAT(pickup_zone.Zone, '/', dropoff_zone.Zone) loc_pair
    FROM
        fhv_data_duration
    LEFT JOIN zone pickup_zone
        ON fhv_data_duration.PULocationID = pickup_zone.LocationID
    LEFT JOIN zone dropoff_zone
        ON fhv_data_duration.DOLocationID = dropoff_zone.LocationID
    ) zoness
GROUP BY
    1
ORDER BY 
    2 DESC
LIMIT 10
""").show()

+--------------------+---------+
|            loc_pair|frequency|
+--------------------+---------+
|East New York/Eas...|    45041|
|Borough Park/Boro...|    37329|
|   Canarsie/Canarsie|    28026|
|Crown Heights Nor...|    25976|
| Bay Ridge/Bay Ridge|    17934|
|     Astoria/Astoria|    14688|
|Jackson Heights/J...|    14688|
|Central Harlem No...|    14481|
|Bushwick South/Bu...|    14424|
|Flatbush/Ditmas P...|    13976|
+--------------------+---------+

